In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install m2cgen


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mtcnn
!pip install keras
!pip install keras==2.4.3
!pip install keras-vggface==0.6
!pip install mtcnn==0.1.0
!pip install tensorflow==2.3.1
!pip install keras_vggface keras_applications 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data={"Ayush_Raj":"/content/drive/MyDrive/Image_for_face_authentication/ayush3.PNG", 
      "Dev_Paliwal":"/content/drive/MyDrive/Image_for_face_authentication/dev1.jpg",
      "Nitin_Yadav":"/content/drive/MyDrive/Image_for_face_authentication/nitin yadav1.jpg",
      "Aman_Gupta":"/content/drive/MyDrive/Image_for_face_authentication/aman1.PNG",
      "Harsh_Tiwari":"/content/drive/MyDrive/Image_for_face_authentication/Harshtiwari3.jpg",
      "Keshav":"/content/drive/MyDrive/Image_for_face_authentication/keshav1.jpg",
      "Asad":"/content/drive/MyDrive/Image_for_face_authentication/Asad1.jpg",
      "Prashant":"/content/drive/MyDrive/Image_for_face_authentication/Prashnat1.jpeg",
      "Harshvajpayee":"/content/drive/MyDrive/Image_for_face_authentication/Harshvajpayee.jpeg"
      }

In [ ]:
Name = input("Enter Your Name ")

In [ ]:
path=data[Name]
print(path)

In [ ]:
from mtcnn.mtcnn  import MTCNN
import cv2
import numpy as np
from keras.layers import Input
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

from scipy.spatial.distance import cosine 

import warnings 
warnings.filterwarnings("ignore",category=FutureWarning)

In [ ]:
image =cv2.imread(path)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(image)

In [ ]:
detector = MTCNN()
real_faces= detector.detect_faces(image)
for face in real_faces:
  print(face)

In [ ]:
def create_box(image):
  faces=detector.detect_faces(image)
  bounding_box=faces[0]['box']

  cv2.rectangle(image,
                (bounding_box[0],bounding_box[1]),
                (bounding_box[0]+bounding_box[2],bounding_box[1]+bounding_box[3]),
                (0,155,255),
                2)
  return image 

In [ ]:
cv2_imshow(create_box(image))

In [ ]:
def extract_face(image,resize=(224,224)):
  image=cv2.imread(image)
  faces=detector.detect_faces(image)
  x1,y1,width,height=faces[0]['box']
  x2,y2=x1+width,y1+height

  face_boundary=image[y1:y2, x1:x2]
  face_image=cv2.resize(face_boundary,resize)
  return face_image

In [ ]:
face_image=extract_face(path)
cv2_imshow(face_image)

In [ ]:
def get_embedding(faces):
  face=np.asarray(faces, 'float32')

  face=preprocess_input(face,version=2)
  model=VGGFace(model='resnet50', include_top=False,input_shape=(224,224,3),pooling='avg')
  return model.predict(face)


In [ ]:

def get_similarity(faces):
  embeddings=get_embedding(faces)
  score= cosine(embeddings[0],embeddings[1])

  if score <=0.4:
    return "face Matched",score
  return "Face Not Matched ", score  

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
def Result():
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
    
    # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

In [ ]:
faces =[ extract_face(image) for image in [path,
                                           take_photo()]]
get_similarity(faces)
